- 22.01.23
- FI 측정할 때, train test split 하지 않고 그대로 사용
- 즉 X,y로 훈련하고 X 에서 피처 바꿔가면서 error 측정

In [6]:
import numpy as np, os
import pandas as pd

# import optuna

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717 
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display



In [7]:
import warnings
warnings.filterwarnings(action='ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

print(tf.__version__)
print(keras.__version__)

2.5.0
2.5.0


In [8]:
import random    
seed_num = 48
random.seed(seed_num)

X = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)

i = round(X.shape[0]*0.8)
X.shape, y.shape

((7727, 10, 4069), (7727,))

In [9]:
COLS = list(pd.read_csv('/project/guri/data/data2d_forposter.csv').columns)
# COLS = COLS[:100]

In [10]:
len(set(COLS))

4069

In [ ]:
seed_num = 48
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X.shape[1],X.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))
#fit
ep = 500
pa = 30
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= "adam", loss = "binary_crossentropy", metrics=['acc'])

In [13]:
# checkpoint_filepath = f"test.hdf5"

# lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
es = EarlyStopping(monitor="val_loss", patience=pa, verbose=1, mode="min", restore_best_weights=True)
# sv = keras.callbacks.ModelCheckpoint(
#     checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
#     save_weights_only=False, mode='auto', save_freq='epoch',
#     options=None
# )
        
history = lstm.fit(X, y, validation_split=0.25, batch_size=128, epochs=ep, callbacks=[es], shuffle=False)

# results = []
# print(' Computing LSTM feature importance...')

# # COMPUTE BASELINE (NO SHUFFLE)
# oof_preds = lstm.predict(X_test, verbose=0)
# baseline_mae = np.mean(np.abs( oof_preds-y_test ))
# results.append({'feature':'BASELINE','baseline_mae':baseline_mae}) 

Epoch 1/500
46/46 [==============================] - 3s 53ms/step - loss: 0.6606 - acc: 0.6475 - val_loss: 0.7468 - val_acc: 0.4865
Epoch 2/500
46/46 [==============================] - 2s 53ms/step - loss: 0.6538 - acc: 0.6514 - val_loss: 0.7343 - val_acc: 0.4865
Epoch 3/500
46/46 [==============================] - 2s 53ms/step - loss: 0.6193 - acc: 0.6533 - val_loss: 0.6492 - val_acc: 0.4865
Epoch 4/500
46/46 [==============================] - 2s 53ms/step - loss: 0.5025 - acc: 0.7724 - val_loss: 0.5709 - val_acc: 0.7133
Epoch 5/500
46/46 [==============================] - 2s 53ms/step - loss: 0.4359 - acc: 0.8091 - val_loss: 0.5550 - val_acc: 0.7469
Epoch 6/500
46/46 [==============================] - 2s 53ms/step - loss: 0.3870 - acc: 0.8450 - val_loss: 0.5670 - val_acc: 0.7464
Epoch 7/500
46/46 [==============================] - 2s 53ms/step - loss: 0.3523 - acc: 0.8702 - val_loss: 0.5843 - val_acc: 0.7453
Epoch 8/500
46/46 [==============================] - 2s 53ms/step - loss: 0.

`tf.distribute.get_strategy()`
- 훈련을 여러 처리 장치들로 분산시키는 것
- gpu 쓰는 것 X

In [12]:
gpu_strategy = tf.distribute.get_strategy()
results = []
with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X[:,:,k].copy()
        np.random.shuffle(X[:,:,k])

        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        oof_preds = lstm.predict(X, verbose=0)
        mae = np.mean(np.abs( oof_preds-y ))

        results.append({'feature':COLS[k],'mae':mae})
        X[:,:,k] = save_col
        # DISPLAY LSTM FEATURE IMPORTANCE
    #     print()
    df = pd.DataFrame(results)
    df = df.sort_values('mae')

  0%|          | 0/4069 [00:00<?, ?it/s]

In [13]:
df.sort_values('mae', ascending=False)

,feature,mae
19,50861,0.484713
3833,63323026201,0.484649
3457,55390000401,0.484618
2190,409490234,0.484565
187,51249,0.484392
...,...,...
41,50893,0.483615
34,50882,0.483545
56,50912,0.483045
121,51006,0.482944


In [19]:
oof_preds = lstm.predict(X, verbose=0)
mae = np.mean(np.abs( oof_preds-y ))
print(mae)
results.append({'feature':'BASELINE','mae':mae})

0.48389368737317645


In [21]:
df = pd.DataFrame(results)
df = df.sort_values('mae')

In [22]:
df.sort_values('mae', ascending=False)

,feature,mae
19,50861,0.484713
3833,63323026201,0.484649
3457,55390000401,0.484618
2190,409490234,0.484565
187,51249,0.484392
...,...,...
41,50893,0.483615
34,50882,0.483545
56,50912,0.483045
121,51006,0.482944


In [23]:
df.to_csv('./nonsplit_FI.csv', index=False)

In [26]:
fi = pd.read_csv('./nonsplit_FI.csv')
fi[fi['feature'] == 'BASELINE']

,feature,mae
28,BASELINE,0.483894


# BinaryCrossEntropy

In [10]:
# def BinaryCrossEntropy(y_true, y_pred):
#     m = y_true.shape[1]
#     y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
#     # Calculating loss
#     loss = -1/m * (np.dot(y_true.T, np.log(y_pred)) + np.dot((1 - y_true).T, np.log(1 - y_pred)))

#     return loss

In [16]:
import time
from sklearn import metrics 
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()

def item_acc(num):
    results = []
    x = X.copy()
    save_col = x[:,:,num].copy()
    np.random.shuffle(x[:,:,num])

    preds = lstm.predict(x)
    loss = bce(y, preds).numpy()
    
    preds[preds>0.5]=1
    preds[preds<=0.5]=0
    acc = metrics.accuracy_score(y, preds)
    
    results.append({'feature':COLS[num],'bce':loss})
    x[:,:,num] = save_col

    print(f'item idx {num} acc & loss :', acc, loss)
    return results

In [ ]:
gpu_strategy = tf.distribute.get_strategy()

total_result = []
with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):
        total_result.append(item_acc(k))

  0%|          | 0/4069 [00:00<?, ?it/s]

item idx 0 acc & loss : 0.8118286527759804 0.99001414
item idx 1 acc & loss : 0.8141581467581209 0.9901495
item idx 2 acc & loss : 0.814287563090462 0.9903321
item idx 3 acc & loss : 0.8140287304257797 0.99014914
item idx 4 acc & loss : 0.814287563090462 0.99013716
item idx 5 acc & loss : 0.8138993140934386 0.9900891
item idx 6 acc & loss : 0.814287563090462 0.99004775
item idx 7 acc & loss : 0.8140287304257797 0.99014693
item idx 8 acc & loss : 0.8150640610845089 0.9903943
item idx 9 acc & loss : 0.8140287304257797 0.99015063
item idx 10 acc & loss : 0.8128639834347094 0.99042976
item idx 11 acc & loss : 0.8137698977610974 0.9903069
item idx 12 acc & loss : 0.8136404814287563 0.9902396
item idx 13 acc & loss : 0.8141581467581209 0.9901827
item idx 14 acc & loss : 0.8144169794228031 0.99014646
item idx 15 acc & loss : 0.8141581467581209 0.9901395
item idx 16 acc & loss : 0.8140287304257797 0.9901525
item idx 17 acc & loss : 0.8138993140934386 0.99014574
item idx 18 acc & loss : 0.81402

item idx 151 acc & loss : 0.8140287304257797 0.990152
item idx 152 acc & loss : 0.8140287304257797 0.99015176
item idx 153 acc & loss : 0.8140287304257797 0.99015266
item idx 154 acc & loss : 0.8140287304257797 0.9901643
item idx 155 acc & loss : 0.814287563090462 0.99003243
item idx 156 acc & loss : 0.8140287304257797 0.99015135
item idx 157 acc & loss : 0.8137698977610974 0.99004394
item idx 158 acc & loss : 0.8140287304257797 0.9901521
item idx 159 acc & loss : 0.8140287304257797 0.9901344
item idx 160 acc & loss : 0.8138993140934386 0.99018586
item idx 161 acc & loss : 0.8140287304257797 0.9901531
item idx 162 acc & loss : 0.8131228160993917 0.9899695
item idx 163 acc & loss : 0.8140287304257797 0.990152
item idx 164 acc & loss : 0.8140287304257797 0.990152
item idx 165 acc & loss : 0.8140287304257797 0.9901521
item idx 166 acc & loss : 0.8140287304257797 0.99015146
item idx 167 acc & loss : 0.8140287304257797 0.9901509
item idx 168 acc & loss : 0.8140287304257797 0.990152
item idx

item idx 300 acc & loss : 0.8140287304257797 0.99013656
item idx 301 acc & loss : 0.8145463957551443 0.99027973
item idx 302 acc & loss : 0.8140287304257797 0.9901517
item idx 303 acc & loss : 0.8138993140934386 0.9901475
item idx 304 acc & loss : 0.8137698977610974 0.9900495
item idx 305 acc & loss : 0.8116992364436392 0.98939645
item idx 306 acc & loss : 0.8141581467581209 0.9899414
item idx 307 acc & loss : 0.8122169017730038 0.98927784
item idx 308 acc & loss : 0.8140287304257797 0.9901527
item idx 309 acc & loss : 0.8140287304257797 0.99013805
item idx 310 acc & loss : 0.8140287304257797 0.99014753
item idx 311 acc & loss : 0.8140287304257797 0.99015146
item idx 312 acc & loss : 0.813381648764074 0.990289
item idx 313 acc & loss : 0.8140287304257797 0.990158
item idx 314 acc & loss : 0.8140287304257797 0.9901381
item idx 315 acc & loss : 0.8140287304257797 0.990152
item idx 316 acc & loss : 0.8140287304257797 0.990158
item idx 317 acc & loss : 0.8140287304257797 0.9901591
item idx

item idx 449 acc & loss : 0.8140287304257797 0.990152
item idx 450 acc & loss : 0.8140287304257797 0.990152
item idx 451 acc & loss : 0.8140287304257797 0.990152
item idx 452 acc & loss : 0.8140287304257797 0.990152
item idx 453 acc & loss : 0.8140287304257797 0.99015397
item idx 454 acc & loss : 0.8140287304257797 0.990152
item idx 455 acc & loss : 0.8140287304257797 0.9901486
item idx 456 acc & loss : 0.8140287304257797 0.9901598
item idx 457 acc & loss : 0.8140287304257797 0.99014705
item idx 458 acc & loss : 0.8140287304257797 0.990152
item idx 459 acc & loss : 0.8140287304257797 0.9901472
item idx 460 acc & loss : 0.8140287304257797 0.9901516
item idx 461 acc & loss : 0.8140287304257797 0.990152
item idx 462 acc & loss : 0.8140287304257797 0.9901509
item idx 463 acc & loss : 0.8140287304257797 0.9901511
item idx 464 acc & loss : 0.8140287304257797 0.990124
item idx 465 acc & loss : 0.8137698977610974 0.9901703
item idx 466 acc & loss : 0.8140287304257797 0.9901477
item idx 467 acc

...

item idx 2803 acc & loss : 0.8140287304257797 0.99015784
item idx 2804 acc & loss : 0.8137698977610974 0.99014497
item idx 2805 acc & loss : 0.8140287304257797 0.990152
item idx 2806 acc & loss : 0.8140287304257797 0.9901524
item idx 2807 acc & loss : 0.8141581467581209 0.9901557
item idx 2808 acc & loss : 0.8138993140934386 0.9900759
item idx 2809 acc & loss : 0.8140287304257797 0.99015176
item idx 2810 acc & loss : 0.8140287304257797 0.990152
item idx 2811 acc & loss : 0.8140287304257797 0.9901507
item idx 2812 acc & loss : 0.8144169794228031 0.9899057
item idx 2813 acc & loss : 0.8140287304257797 0.990152
item idx 2814 acc & loss : 0.8140287304257797 0.990152
item idx 2815 acc & loss : 0.8140287304257797 0.9901501
item idx 2816 acc & loss : 0.8140287304257797 0.9901519
item idx 2817 acc & loss : 0.8140287304257797 0.990152
item idx 2818 acc & loss : 0.8141581467581209 0.990133
item idx 2819 acc & loss : 0.8140287304257797 0.990152
item idx 2820 acc & loss : 0.8140287304257797 0.9901

item idx 2950 acc & loss : 0.8141581467581209 0.99017876
item idx 2951 acc & loss : 0.8140287304257797 0.9901319
item idx 2952 acc & loss : 0.8140287304257797 0.9901517
item idx 2953 acc & loss : 0.8141581467581209 0.9901757
item idx 2954 acc & loss : 0.8140287304257797 0.9901499
item idx 2955 acc & loss : 0.8140287304257797 0.9901426
item idx 2956 acc & loss : 0.8138993140934386 0.99013495
item idx 2957 acc & loss : 0.8140287304257797 0.99012595
item idx 2958 acc & loss : 0.8140287304257797 0.9901527
item idx 2959 acc & loss : 0.8137698977610974 0.9901738
item idx 2960 acc & loss : 0.8140287304257797 0.9900513
item idx 2961 acc & loss : 0.8140287304257797 0.9901511
item idx 2962 acc & loss : 0.8137698977610974 0.9901006
item idx 2963 acc & loss : 0.8140287304257797 0.9901886
item idx 2964 acc & loss : 0.8140287304257797 0.9901505
item idx 2965 acc & loss : 0.8140287304257797 0.9901775
item idx 2966 acc & loss : 0.8140287304257797 0.99015176
item idx 2967 acc & loss : 0.814028730425779

item idx 3097 acc & loss : 0.8140287304257797 0.990152
item idx 3098 acc & loss : 0.8140287304257797 0.99015224
item idx 3099 acc & loss : 0.814287563090462 0.99012005
item idx 3100 acc & loss : 0.8140287304257797 0.98997515
item idx 3101 acc & loss : 0.8140287304257797 0.9901546
item idx 3102 acc & loss : 0.8140287304257797 0.99014914
item idx 3103 acc & loss : 0.8140287304257797 0.990152
item idx 3104 acc & loss : 0.8138993140934386 0.9901127
item idx 3105 acc & loss : 0.8140287304257797 0.9901527
item idx 3106 acc & loss : 0.8138993140934386 0.99017346
item idx 3107 acc & loss : 0.8140287304257797 0.9901443
item idx 3108 acc & loss : 0.8140287304257797 0.99012846
item idx 3109 acc & loss : 0.8140287304257797 0.9901516
item idx 3110 acc & loss : 0.8141581467581209 0.9901119
item idx 3111 acc & loss : 0.8140287304257797 0.9901457
item idx 3112 acc & loss : 0.8141581467581209 0.99013454
item idx 3113 acc & loss : 0.8140287304257797 0.9901471
item idx 3114 acc & loss : 0.812863983434709

item idx 3243 acc & loss : 0.8140287304257797 0.9901182
item idx 3244 acc & loss : 0.8140287304257797 0.9901231
item idx 3245 acc & loss : 0.8140287304257797 0.99014866
item idx 3246 acc & loss : 0.8140287304257797 0.990143
item idx 3247 acc & loss : 0.8138993140934386 0.98979515
item idx 3248 acc & loss : 0.8136404814287563 0.9900821
item idx 3249 acc & loss : 0.8140287304257797 0.99014044
item idx 3250 acc & loss : 0.8141581467581209 0.99020946
item idx 3251 acc & loss : 0.814287563090462 0.9901237
item idx 3252 acc & loss : 0.813381648764074 0.99001
item idx 3253 acc & loss : 0.8140287304257797 0.9901569
item idx 3254 acc & loss : 0.8140287304257797 0.99015385
item idx 3255 acc & loss : 0.8140287304257797 0.9900755
item idx 3256 acc & loss : 0.8140287304257797 0.99013066
item idx 3257 acc & loss : 0.8140287304257797 0.99014705
item idx 3258 acc & loss : 0.8140287304257797 0.99014795
item idx 3259 acc & loss : 0.8140287304257797 0.9901523
item idx 3260 acc & loss : 0.8135110650964151

item idx 3389 acc & loss : 0.8140287304257797 0.99014866
item idx 3390 acc & loss : 0.8140287304257797 0.9901589
item idx 3391 acc & loss : 0.8140287304257797 0.99015325
item idx 3392 acc & loss : 0.8140287304257797 0.990152
item idx 3393 acc & loss : 0.8140287304257797 0.99013865
item idx 3394 acc & loss : 0.8138993140934386 0.99015343
item idx 3395 acc & loss : 0.8140287304257797 0.990152
item idx 3396 acc & loss : 0.8140287304257797 0.9901494
item idx 3397 acc & loss : 0.8140287304257797 0.9901062
item idx 3398 acc & loss : 0.8140287304257797 0.990152
item idx 3399 acc & loss : 0.8140287304257797 0.9901457
item idx 3400 acc & loss : 0.8140287304257797 0.9901185
item idx 3401 acc & loss : 0.8140287304257797 0.9901563
item idx 3402 acc & loss : 0.8140287304257797 0.9901516
item idx 3403 acc & loss : 0.8138993140934386 0.99015844
item idx 3404 acc & loss : 0.8140287304257797 0.990152
item idx 3405 acc & loss : 0.8140287304257797 0.9901518
item idx 3406 acc & loss : 0.8140287304257797 0

In [ ]:
df1 = pd.DataFrame(total_result)
df1 = df.sort_values('bce')
df1.to_csv('./nonsplit_bceFI.csv', index=False)
df1

In [18]:
total_result

[[{'feature': '0', 'bce': 0.9416786}],
 [{'feature': '50803', 'bce': 0.94078207}],
 [{'feature': '50804', 'bce': 0.94061154}],
 [{'feature': '50805', 'bce': 0.94079375}],
 [{'feature': '50806', 'bce': 0.9408754}],
 [{'feature': '50808', 'bce': 0.9408267}],
 [{'feature': '50809', 'bce': 0.9406753}],
 [{'feature': '50811', 'bce': 0.9407451}],
 [{'feature': '50813', 'bce': 0.9402641}],
 [{'feature': '50814', 'bce': 0.94079375}],
 [{'feature': '50818', 'bce': 0.94062287}],
 [{'feature': '50820', 'bce': 0.94067603}],
 [{'feature': '50821', 'bce': 0.9405592}],
 [{'feature': '50822', 'bce': 0.94086486}],
 [{'feature': '50824', 'bce': 0.94072884}],
 [{'feature': '50852', 'bce': 0.94078016}],
 [{'feature': '50853', 'bce': 0.94079155}],
 [{'feature': '50856', 'bce': 0.94079363}],
 [{'feature': '50858', 'bce': 0.94079363}],
 [{'feature': '50861', 'bce': 0.9408003}],
 [{'feature': '50862', 'bce': 0.9412688}],
 [{'feature': '50863', 'bce': 0.9408634}],
 [{'feature': '50864', 'bce': 0.94079363}],
 [